In [81]:
import pandas as pd
from pathlib import Path
import re

import plotly.graph_objs as go
import plotly.express as px

### Solution analysis
This notebooks generates tables and plots for the scenario analysis in the paper.

In [82]:
# Inputs and outputs
results_path = Path("../results")

tables_path = Path("tables")
figures_path = Path("figures")

In [83]:
# Load all displayed cases
cases = [
    "baseline_2019", "today_2024", "future_2030",
    "policy_co2_100EUR", "policy_co2_200EUR", "policy_co2_500EUR", "policy_co2_750EUR","policy_co2_1000EUR", "policy_co2_1500EUR", "policy_co2_2000EUR",
    "policy_thermic_20pct", "policy_thermic_50pct", "policy_thermic_75pct", "policy_thermic_100pct",
    "policy_elez", "policy_electric", "policy_cost_parity" # , "policy_cost_parity_high_range"
]

df_cases = []

for case in cases:
    df_partial = pd.read_parquet(results_path / "scenario_solutions/{}.vehicles.parquet".format(case))
    f_electric = df_partial["vehicle_type"].str.endswith("electric")

    df_cases.append({
        "case": case,

        "parcels": df_partial.loc[:, "deliveries"].sum(),
        "thermic_parcels": df_partial.loc[~f_electric, "deliveries"].sum(),
        "electric_parcels": df_partial.loc[f_electric, "deliveries"].sum(),

        "vehicles": len(df_partial[["vehicle_id", "depot", "operator"]].drop_duplicates()),
        "thermic_vehicles": len(df_partial[~f_electric][["vehicle_id", "depot", "operator"]].drop_duplicates()),
        "electric_vehicles": len(df_partial[f_electric][["vehicle_id", "depot", "operator"]].drop_duplicates()),
        
        "distance_km": df_partial["distance_m"].sum() * 1e-3,
        "thermic_distance_km": df_partial[~f_electric]["distance_m"].sum() * 1e-3,
        "electric_distance_km": df_partial[f_electric]["distance_m"].sum() * 1e-3,

        "fuel_L": df_partial["fuel_L"].sum(),
        "electricity_kWh": df_partial["electricity_Wh"].sum() * 1e-3,
        "energy_kWh": df_partial["energy_Wh"].sum() * 1e-3,
        "co2eq_kg": df_partial["co2eq_g"].sum() * 1e-3,

        "total_cost_EUR": df_partial["total_cost_EUR"].sum(),

        "deliveries": df_partial["deliveries"].sum()
    })

df_cases = pd.DataFrame.from_records(df_cases)
df_cases["cost_per_delivery_EUR"] = df_cases["total_cost_EUR"] / df_cases["deliveries"]
df_cases["cost_per_distance_EUR_km"] = df_cases["total_cost_EUR"] / df_cases["distance_km"]

df_cases = df_cases.drop(columns = [
    "deliveries"
])

In [84]:
# Prepare table on BAU results
df_table = df_cases.copy()
df_table = df_table[
    df_table["case"].isin([
        "baseline_2019", "today_2024", "future_2030"
    ])
]

df_table["case"] = df_table["case"].replace({ 
    "baseline_2019": "Baseline 2019",
    "today_2024": "Today 2024",
    "future_2030": "Future 2030"
})

df_table = df_table[[
    "case",
    "parcels",
    "thermic_parcels",
    "electric_parcels",
    "vehicles",
    "thermic_vehicles",
    "electric_vehicles",
    "distance_km",
    "thermic_distance_km",
    "electric_distance_km",
    "total_cost_EUR",
    "cost_per_delivery_EUR",
    "cost_per_distance_EUR_km",
    "fuel_L",
    "electricity_kWh",
    "energy_kWh",
    "co2eq_kg",
]]

df_table = df_table.rename(columns = {
    "parcels": "Delivered parcels",
    "thermic_parcels": "\\hspace{0.2cm}ICV",
    "electric_parcels": "\\hspace{0.2cm}BEV",
    "vehicles": "Vehicles",
    "thermic_vehicles": "\\hspace{0.2cm}ICV",
    "electric_vehicles": "\\hspace{0.2cm}BEV",
    "distance_km": "Distance [km]",
    "thermic_distance_km": "\\hspace{0.2cm}ICV",
    "electric_distance_km": "\\hspace{0.2cm}BEV",
    "fuel_L": "Fuel [L]",
    "electricity_kWh": "Electricity [kWh]",
    "energy_kWh": "Energy [kWh]",
    "co2eq_kg": "Emissions [kgCO2eq]",
    "total_cost_EUR": "Cost per day [EUR]",
    "cost_per_delivery_EUR": "Cost per shipment [EUR]",
    "cost_per_distance_EUR_km": "Cost per kilometer [EUR]",
})

df_table.columns = [
    re.sub(r"\[(.+?)\]", "\\\\footnotesize{[\\1]}", c)
    for c in df_table.columns
]

df_table = df_table.set_index("case").T

df_table = df_table.style.format(thousands = ",", precision = 0) # .hide(axis = "index")

selector = (
    df_table.index.str.startswith("Cost per shipment") | df_table.index.str.startswith("Cost per kilo"), 
    df_table.columns
)

df_table = df_table.format(precision = 2, subset = selector)

with open(tables_path / "baseline.tex", "w+") as f:
    table = df_table.to_latex(
        caption = "Business-as-usual results", label = "tab:bau_results", hrules = True)
    #table = table.replace("$\\sum$", "\\midrule\n$\\sum$")
    table = table.replace("\\begin{table}", "\\begin{table}\n\\centering")

    table = table.replace("Vehicles", "\\midrule\nVehicles")
    table = table.replace("Distance", "\\midrule\nDistance")
    table = table.replace("Fuel", "\\midrule\nFuel")
    table = table.replace("Energy", "\\midrule\nEnergy")
    table = table.replace("Cost per day", "\\midrule\nCost per day")
    table = table.replace("case", "")

    f.write(table)

df_table

case,Baseline 2019,Today 2024,Future 2030
Delivered parcels,"26,984","35,986","54,527"
\hspace{0.2cm}ICV,"22,410","31,904","51,489"
\hspace{0.2cm}BEV,"4,574","4,082","3,038"
Vehicles,350,436,610
\hspace{0.2cm}ICV,283,378,568
\hspace{0.2cm}BEV,67,58,42
Distance \footnotesize{[km]},"16,571","19,783","24,987"
\hspace{0.2cm}ICV,"10,696","14,508","21,477"
\hspace{0.2cm}BEV,"5,875","5,275","3,510"
Cost per day \footnotesize{[EUR]},"41,438","52,119","73,492"


In [85]:
# Prepare key plot of the paper showing the BAU and policy scenarios
colors = px.colors.qualitative.Plotly
co2_scale = 1e-3 # 1e-3 * 202

df_co2 = df_cases[df_cases["case"].str.contains("co2")].copy()
df_co2["price"] = df_co2["case"].apply(lambda x: x.split("_")[-1])

df_thermic = df_cases[df_cases["case"].str.contains("thermic")].copy()
df_thermic["tax"] = df_thermic["case"].apply(lambda x: "+" + x.split("_")[-1].replace("pct", "%"))

df_2030 = df_cases[df_cases["case"] == "future_2030"]
co2_2030 = df_2030["co2eq_kg"].values[0]
cost_2030 = df_2030["cost_per_delivery_EUR"].values[0]

df_2019 = df_cases[df_cases["case"] == "baseline_2019"]
co2_2019 = df_2019["co2eq_kg"].values[0]
cost_2019 = df_2019["cost_per_delivery_EUR"].values[0]

df_2024 = df_cases[df_cases["case"] == "today_2024"]
co2_2024 = df_2024["co2eq_kg"].values[0]
cost_2024 = df_2024["cost_per_delivery_EUR"].values[0]

df_qualitative = df_cases[df_cases["case"].isin([
    "policy_elez", "policy_electric", "policy_cost_parity", "policy_cost_parity_high_range"
])].sort_values(by = "case")

figure = go.Figure()

#figure.add_trace(go.Scatter(
#    x = [df_2019["co2eq_kg"].values[0] * co2_scale],
#    y = [1.51],
#    mode = "lines+text", text = [" 2019 Emissions  <br />({:.2f} EUR)  ".format(cost_2024)],
#    line_color = "#AAAAAA", textposition = "middle left",
#    showlegend = False, textfont = dict(color = "#AAAAAA")
#))

#figure.add_trace(go.Scatter(
#    x = [df_2019["co2eq_kg"].values[0] * co2_scale] * 2,
#    y = [0.0, 1.65],
#    mode = "lines",
#    line_color = "#AAAAAA", line_dash = "dot",
#    showlegend = False
#))

figure.add_trace(go.Scatter(
    x = [co2_2019 * co2_scale, co2_2024 * co2_scale, co2_2030 * co2_scale],
    y = [cost_2019, cost_2024, cost_2030], line_color = "black",
    mode="lines+markers+text",
    text = ["2019", "2024", "2030"], textposition = "middle right",
    name = "BAU"
))

figure.add_trace(go.Scatter(
    x = [co2_2030 * co2_scale] + list(df_co2["co2eq_kg"] * co2_scale),
    y = [cost_2030] + list(df_co2["cost_per_delivery_EUR"]),
    mode="lines+markers+text",
    text = [""] + list(df_co2["price"]), textposition = "top right",
    name = "Carbon tax", line_color = colors[0], textfont = dict(color = colors[0])
))

figure.add_trace(go.Scatter(
    x = [co2_2030 * co2_scale] + list(df_thermic["co2eq_kg"] * co2_scale),
    y = [cost_2030] + list(df_thermic["cost_per_delivery_EUR"]),
    mode="lines+markers+text",
    text = [""] + list(df_thermic["tax"]), textposition = "bottom center",
    name = "ICV tax", line_color = colors[1], textfont = dict(color = colors[1])
))

figure.add_trace(go.Scatter(
    x = df_qualitative["co2eq_kg"] * co2_scale,
    y = df_qualitative["cost_per_delivery_EUR"],
    mode="markers+text",
    text = ["Cost parity", "Full Electric", "eLEZ"], textposition = ["top right", "top right", "bottom center"],
    name = "Qualitative", marker = dict(size = 7), line_color = colors[2],
    textfont = dict(color = colors[2])
))

figure.update_layout(
    xaxis_title = "Daily emissions [tCO2eq]",
    yaxis_title = "Cost per parcel [EUR]",
    yaxis_range = [1.3, 1.55],
    xaxis_range = [0.5, 4.78],
    width = 600, height = 300,
    margin = dict(l = 0, r = 0, b = 0, t = 15, pad = 0)
)

figure.write_image(figures_path / "scenario_results.pdf")
figure

In [86]:
# Prepare runtime information plot
df_runtime = []

for case in cases:
    df_partial = pd.read_parquet(results_path / "scenario_solutions/{}.instances.parquet".format(case))
    df_partial = df_partial[["deliveries", "runtime:writing", "runtime:reading", "runtime:vroom_loading", "runtime:vroom_solving", "retries"]]
    df_runtime.append(df_partial)

df_runtime = pd.concat(df_runtime)
df_runtime["retries"] = df_runtime["retries"].astype(str)

df_runtime["runtime:vroom_solving"] *= 1e-3 
df_runtime["runtime:vroom_solving"] /= 3600

df_runtime = df_runtime.rename(columns = {
    "retries": "Retries",
    "runtime:vroom_solving": "Solution time [h]",
    "deliveries": "Shipments"
})

figure = px.scatter(df_runtime, x = "Shipments", y = "Solution time [h]", color = "Retries")

figure.update_layout(
    width = 600, height = 200,
    margin = dict(l = 0, r = 0, b = 0, t = 0, pad = 0)
)

figure.write_image(figures_path / "runtime.pdf")
figure